!pip3 install numpy pandas tqdm sklearn tensorflow pyaudio librosa

# Importing the required libraries

import pandas as pd
import numpy as np
import os
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
import pyaudio
import os
import wave
import librosa
import numpy as np
from sys import byteorder
from array import array
from struct import pack


In [3]:
import pandas as pd
import numpy as np
import os
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split

## Reading the csv file

In [4]:
df = pd.read_csv("balanced-all.csv")
df.head()

,filename,gender
0,data/cv-other-train/sample-069205.npy,female
1,data/cv-valid-train/sample-063134.npy,female
2,data/cv-other-train/sample-080873.npy,female
3,data/cv-other-train/sample-105595.npy,female
4,data/cv-valid-train/sample-144613.npy,female


In [5]:
df.shape


(66938, 2)

In [6]:
df.groupby("gender").count()

,filename
gender,
female,33469
male,33469


In [7]:
df.gender.value_counts()

gender
female    33469
male      33469
Name: count, dtype: int64

In [8]:
# get total samples
n_samples = len(df)
# get total male samples
n_male_samples = len(df[df['gender'] == 'male'])
# get total female samples
n_female_samples = len(df[df['gender'] == 'female'])
print("Total samples:", n_samples)
print("Total male samples:", n_male_samples)
print("Total female samples:", n_female_samples)

Total samples: 66938
Total male samples: 33469
Total female samples: 33469


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66938 entries, 0 to 66937
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  66938 non-null  object
 1   gender    66938 non-null  object
dtypes: object(2)
memory usage: 1.0+ MB


## Creating the labels form

In [10]:

label2int = {
    "male": 1,
    "female": 0
}


def load_data(vector_length=128):
    """A function to load gender recognition dataset from `data` folder
    After the second run, this will load from results/features.npy and results/labels.npy files
    as it is much faster!"""
    # make sure results folder exists
    if not os.path.isdir("results"):
        os.mkdir("results")
    # if features & labels already loaded individually and bundled, load them from there instead
    if os.path.isfile("results/features.npy") and os.path.isfile("results/labels.npy"):
        X = np.load("results/features.npy")
        y = np.load("results/labels.npy")
        return X, y
    # read dataframe
    df = pd.read_csv("balanced-all.csv")
    # get total samples
    n_samples = len(df)
    # get total male samples
    n_male_samples = len(df[df['gender'] == 'male'])
    # get total female samples
    n_female_samples = len(df[df['gender'] == 'female'])
    print("Total samples:", n_samples)
    print("Total male samples:", n_male_samples)
    print("Total female samples:", n_female_samples)
    # initialize an empty array for all audio features
    X = np.zeros((n_samples, vector_length))
    # initialize an empty array for all audio labels (1 for male and 0 for female)
    y = np.zeros((n_samples, 1))
    for i, (filename, gender) in tqdm.tqdm(enumerate(zip(df['filename'], df['gender'])), "Loading data", total=n_samples):
        features = np.load(filename)
        X[i] = features
        y[i] = label2int[gender]
    # save the audio features and labels into files
    # so we won't load each one of them next run
    np.save("results/features", X)
    np.save("results/labels", y)
    return X, y


def split_data(X, y, test_size=0.1, valid_size=0.1):
    # split training set and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=7)
    # split training set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }


def create_model(vector_length=128):
    """5 hidden dense layers from 256 units to 64, not the best model, but not bad."""
    ''' model = Sequential()
    #model.add(Dense(128, input_shape=(vector_length,)))
    model.add(Dense(128, input_shape=(vector_length,)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    # one output neuron with sigmoid activation function, 0 means female, 1 means male
    model.add(Dense(1, activation="sigmoid"))
    # using binary crossentropy as it's male/female classification (binary)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    # print summary of the model
    model.summary()
    return model''' """Updated model with dropout and an input layer of size 128."""
    model = Sequential()
    model.add(Dense(128, input_shape=(vector_length,)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    model.summary()
    return model


    
   

In [11]:
import pandas as pd
import numpy as np
import os
import tqdm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split


label2int = {
    "male": 1,
    "female": 0
}


def load_data(vector_length=128):
    """A function to load gender recognition dataset from `data` folder
    After the second run, this will load from results/features.npy and results/labels.npy files
    as it is much faster!"""
    # make sure results folder exists
    if not os.path.isdir("results"):
        os.mkdir("results")
    # if features & labels already loaded individually and bundled, load them from there instead
    if os.path.isfile("results/features.npy") and os.path.isfile("results/labels.npy"):
        X = np.load("results/features.npy")
        y = np.load("results/labels.npy")
        return X, y
    # read dataframe
    df = pd.read_csv("balanced-all.csv")
    # get total samples
    n_samples = len(df)
    # get total male samples
    n_male_samples = len(df[df['gender'] == 'male'])
    # get total female samples
    n_female_samples = len(df[df['gender'] == 'female'])
    print("Total samples:", n_samples)
    print("Total male samples:", n_male_samples)
    print("Total female samples:", n_female_samples)
    # initialize an empty array for all audio features
    X = np.zeros((n_samples, vector_length))
    # initialize an empty array for all audio labels (1 for male and 0 for female)
    y = np.zeros((n_samples, 1))
    for i, (filename, gender) in tqdm.tqdm(enumerate(zip(df['filename'], df['gender'])), "Loading data", total=n_samples):
        features = np.load(filename)
        X[i] = features
        y[i] = label2int[gender]
    # save the audio features and labels into files
    # so we won't load each one of them next run
    np.save("results/features", X)
    np.save("results/labels", y)
    return X, y


def split_data(X, y, test_size=0.1, valid_size=0.1):
    # split training set and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=7)
    # split training set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=valid_size, random_state=7)
    # return a dictionary of values
    return {
        "X_train": X_train,
        "X_valid": X_valid,
        "X_test": X_test,
        "y_train": y_train,
        "y_valid": y_valid,
        "y_test": y_test
    }


def create_model(vector_length=128):
    """5 hidden dense layers from 256 units to 64, not the best model, but not bad."""
    model = Sequential()
    model.add(Dense(128, input_shape=(vector_length,)))
    model.add(Dropout(0.3))
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3))
    # one output neuron with sigmoid activation function, 0 means female, 1 means male
    model.add(Dense(1, activation="sigmoid"))
    # using binary crossentropy as it's male/female classification (binary)
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer="adam")
    # print summary of the model
    model.summary()
    return model

In [12]:
X = np.load("data/cv-other-train/sample-069205.npy")
len(X)

128

In [90]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping

import keras
#from utils import load_data, split_data, create_model

# load the dataset
X, y = load_data()
# split the data into training, validation and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)
# construct the model
model = create_model()

# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100

# train the model using the training set and validating using validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# save the model to a file
model.save("results_try3/model.h5")

# evaluating the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 128)               16512     
                                                                 
 dropout_61 (Dropout)        (None, 128)               0         
                                                                 
 dense_75 (Dense)            (None, 256)               33024     
                                                                 
 dropout_62 (Dropout)        (None, 256)               0         
                                                                 
 dense_76 (Dense)            (None, 128)               32896     
                                                                 
 dropout_63 (Dropout)        (None, 128)               0         
                                                                 
 dense_77 (Dense)            (None, 128)             

In [8]:
# save the model to a file
model.save("results/model.h5")

In [93]:
data['X_test'][0].shape

(128,)

In [241]:
%tensorboard
%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir ="/Users/nanda/Documents/mini_project/logs"--port 0


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
Error: A logdir or db must be specified. For example `tensorboard --logdir mylogdir` or `tensorboard --db sqlite:~/.tensorboard.db`. Run `tensorboard --helpfull` for details and examples.

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-

In [13]:
from tensorboard import program

# Set the log directory where your log files are stored
logdir = "/Users/nanda/Documents/mini_project/logs"

# Set the port number for TensorBoard
port = 0

# Initialize TensorBoard program with the specified log directory
tb = program.TensorBoard()
tb.configure(logdir=logdir, port=port)

# Start TensorBoard server
tb.launch()


'http://localhost:63549/'

import matplotlib.pyplot as plt

# Assuming you have the following variables:
 # List of epoch numbers
  # List of corresponding loss values
  # List of corresponding accuracy values

# Plotting the graph for epochs vs. loss
plt.figure(figsize=(8, 6))
plt.plot(epochs, loss, marker='o', linestyle='-', color='b')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Epochs vs. Loss')
plt.grid(True)
plt.show()

# Plotting the graph for epochs vs. accuracy
plt.figure(figsize=(8, 6))
plt.plot(epochs, accuracy, marker='o', linestyle='-', color='r')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Epochs vs. Accuracy')
plt.grid(True)
plt.show()


import pyaudio
import os
import wave
import librosa
import numpy as np
from sys import byteorder
from array import array
from struct import pack


THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.
    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()



def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result


if __name__ == "__main__":
    # load the saved model (after training)
    # model = pickle.load(open("result/mlp_classifier.model", "rb"))
    from my_utils import load_data, split_data, create_model
    import argparse
    parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
    parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
    args = parser.parse_args()
    file = args.file
    # construct the model
    model = create_model()
    # load the saved/trained weights
    model.load_weights("results_try1/model.h5")
    if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        print("Please talk")
        # put the file name here
        file = "test.wav"
        # record the file (start talking)
        record_to_file(file)
    # extract features and reshape it
    features = extract_feature(file, mel=True).reshape(1, -1)
    # predict the gender!
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    print("Result:", gender)
    print(f"Probabilities:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")

In [14]:
import pyaudio
import os
import wave
import librosa
import numpy as np
from sys import byteorder
from array import array
from struct import pack


THRESHOLD = 500
CHUNK_SIZE = 1024
FORMAT = pyaudio.paInt16
RATE = 16000

SILENCE = 30

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r

In [15]:
def trim(snd_data):
    "Trim the blank spots at the start and end"
    def _trim(snd_data):
        snd_started = False
        r = array('h')

        for i in snd_data:
            if not snd_started and abs(i)>THRESHOLD:
                snd_started = True
                r.append(i)

            elif snd_started:
                r.append(i)
        return r

    # Trim to the left
    snd_data = _trim(snd_data)

    # Trim to the right
    snd_data.reverse()
    snd_data = _trim(snd_data)
    snd_data.reverse()
    return snd_data

def add_silence(snd_data, seconds):
    "Add silence to the start and end of 'snd_data' of length 'seconds' (float)"
    r = array('h', [0 for i in range(int(seconds*RATE))])
    r.extend(snd_data)
    r.extend([0 for i in range(int(seconds*RATE))])
    return r

In [16]:
def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.
    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.5 seconds of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > SILENCE:
            break

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    r = trim(r)
    r = add_silence(r, 0.5)
    return sample_width, r


In [79]:
def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    print("Recording Started")
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
    return path


In [81]:
import numpy as np
import pyaudio
import wave
import librosa

def record_to_file(file_name, duration=5, sample_rate=44100):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=CHUNK)

    frames = []
    for i in range(0, int(sample_rate / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    print('#######')
    wf.close()
    return file_name

def extract_feature(file_name, **kwargs):
    X, sample_rate = librosa.load(file_name)

    features = []
    if kwargs.get("mel"):
        mel = librosa.feature.melspectrogram(y=X, sr=sample_rate)
        print(len(mel))
        features.append(mel)
    if kwargs.get("chroma"):
        chroma = librosa.feature.chroma_stft(y=X, sr=sample_rate)
        features.append(chroma)
    if kwargs.get("contrast"):
        contrast = librosa.feature.spectral_contrast(y=X, sr=sample_rate)
        features.append(contrast)
    if kwargs.get("tonnetz"):
        tonnetz = librosa.feature.tonnetz(y=X, sr=sample_rate)
        features.append(tonnetz)

    if len(features) > 0:
        features = np.concatenate(features, axis=0)

    return features

# Example usage

duration = 8

# Record audio
#record_to_file(file, duration=duration)

# Extract features and reshape
#features = extract_feature(file, mel=True).reshape(-1, 1)

# Perform further processing on the extracted features
# ...



# Rest of the code remains the same...


if __name__ == "__main__":
    # load the saved model (after training)
    # model = pickle.load(open("result/mlp_classifier.model", "rb"))
    from my_utils import load_data, split_data, create_model
    import argparse
    parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
    parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
    args = parser.parse_args()
    #file = "New Recording 17.wav"
    #file = args.file
    # construct the model
    model = create_model()
    # load the saved/trained weights
    file = "New Recording 17.wav"
    model.load_weights("results/model.h5")
   # if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please start speaking...")
        # wait for user to press enter to start recording
        #input("Press Enter to start recording...")
        # specify the duration of the recording (in seconds)
        #duration = 5
        # record the file
        #record_to_file(file, duration=duration)
    # extract features and reshape it
    #if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please talk")
        # put the file name here
        #file = "test.wav"
        # record the file (start talking)
        #record_to_file("/Users/nanda/Documents/mini_project")
    
    #.reshape(1, -1)

# Assuming you have extracted the features into the 'features' variable with shape (None, 27648)

# Reshape the features to match the expected input shape
   # reshaped_features = np.reshape(features, (features.shape[0], 128))

# Now 'reshaped_features' has the shape (None, 128) which matches the model's expected input shape
    #file =("/Users/nanda/Documents/mini_project/recorded12.wav")
    features = extract_feature(file, mel=True,mfcc = True)
    print((features.shape), '******')
    features = features.reshape(-1, 128)
    print(len(features.shape), '******')
    print(type(features))
    # predict the gender!
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    print(f"{file}")
    print("Result:", gender)
    print(f"Probabilities:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")
    print(file)

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_86 (Dense)            (None, 256)               33024     
                                                                 
 dropout_70 (Dropout)        (None, 256)               0         
                                                                 
 dense_87 (Dense)            (None, 256)               65792     
                                                                 
 dropout_71 (Dropout)        (None, 256)               0         
                                                                 
 dense_88 (Dense)            (None, 128)               32896     
                                                                 
 dropout_72 (Dropout)        (None, 128)               0         
                                                                 
 dense_89 (Dense)            (None, 128)             

In [116]:
file = record_to_file("/Users/nanda/Documents/mini_project/recordedmale26.wav")
print(file)

#######
/Users/nanda/Documents/mini_project/recordedmale26.wav


In [121]:
file = "recorded10.wav"
features = extract_feature(file, mel=True)
print((features.shape), '******')
features = features.reshape(-1, 128)
print(len(features.shape), '******')
print(type(features))
    # predict the gender!
male_prob = model.predict(features)[0][0]
female_prob = 1 - male_prob
gender = "male" if male_prob > female_prob else "female"
# show the result!
print(f"{file}")
print("Result:", gender)
print(f"Probabilities:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")
print(file)

128
(128, 216) ******
2 ******
<class 'numpy.ndarray'>
7/7 [==============================] - 0s 3ms/step
recorded10.wav
Result: female
Probabilities:     Male: 37.32%    Female: 62.68%
recorded10.wav


In [122]:
ipd.Audio("recorded6.wav")

In [167]:
def record_to_file(path):
    "Records from the microphone and outputs the resulting data to 'path'"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()
    
record_to_file()

TypeError: record_to_file() missing 1 required positional argument: 'path'

In [318]:
from pydub import AudioSegment
from pydub.playback import play

# Load the audio file
audio = AudioSegment.from_file("/Users/nanda/Documents/mini_project/recorded11.wav", format="wav")

# Play the audio
play(audio)


## Function for pre processing the data files 

In [ ]:
import glob
import os
import pandas as pd
import numpy as np
import shutil
import librosa
from tqdm import tqdm


def extract_feature(file_name, **kwargs):
    """
    Extract feature from audio file `file_name`
        Features supported:
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        e.g:
        `features = extract_feature(path, mel=True, mfcc=True)`
    """
    mfcc = kwargs.get("mfcc")
    chroma = kwargs.get("chroma")
    mel = kwargs.get("mel")
    contrast = kwargs.get("contrast")
    tonnetz = kwargs.get("tonnetz")
    X, sample_rate = librosa.core.load(file_name)
    if chroma or contrast:
        stft = np.abs(librosa.stft(X))
    result = np.array([])
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
        result = np.hstack((result, mfccs))
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, chroma))
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        result = np.hstack((result, mel))
    if contrast:
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        result = np.hstack((result, contrast))
    if tonnetz:
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
        result = np.hstack((result, tonnetz))
    return result

dirname = "data"

if not os.path.isdir(dirname):
    os.mkdir(dirname)


csv_files = glob.glob("*.csv")

for j, csv_file in enumerate(csv_files):
    print("[+] Preprocessing", csv_file)
    df = pd.read_csv(csv_file)
    # only take filename and gender columns
    new_df = df[["filename", "gender"]]
    print("Previously:", len(new_df), "rows")
    # take only male & female genders (i.e droping NaNs & 'other' gender)
    new_df = new_df[np.logical_or(new_df['gender'] == 'female', new_df['gender'] == 'male')]
    print("Now:", len(new_df), "rows")
    new_csv_file = os.path.join(dirname, csv_file)
    # save new preprocessed CSV 
    new_df.to_csv(new_csv_file, index=False)
    # get the folder name
    folder_name, _ = csv_file.split(".")
    audio_files = glob.glob(f"{folder_name}/{folder_name}/*")
    all_audio_filenames = set(new_df["filename"])
    for i, audio_file in tqdm(list(enumerate(audio_files)), f"Extracting features of {folder_name}"):
        splited = os.path.split(audio_file)
        # audio_filename = os.path.join(os.path.split(splited[0])[-1], splited[-1])
        audio_filename = f"{os.path.split(splited[0])[-1]}/{splited[-1]}"
        # print("audio_filename:", audio_filename)
        if audio_filename in all_audio_filenames:
            # print("Copyying", audio_filename, "...")
            src_path = f"{folder_name}/{audio_filename}"
            target_path = f"{dirname}/{audio_filename}"
            #create that folder if it doesn't exist
            if not os.path.isdir(os.path.dirname(target_path)):
                os.mkdir(os.path.dirname(target_path))
            features = extract_feature(src_path, mel=True)
            target_filename = target_path.split(".")[0]
            np.save(target_filename, features)
            # shutil.copyfile(src_path, target_path)

In [35]:
input_shape = (128, )
# Assuming your input data is stored in a variable called 'X'
# Reshape the input data to match the expected shape


X, y = load_data()
# split the data into training, validation and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)
# construct the model
model = create_cnn_model(X)

# use tensorboard to view metrics
tensorboard = TensorBoard(log_dir="logs")
# define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100

# train the model using the training set and validating using validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# save the model to a file
model.save("results/model.h5")

# evaluating the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")

TypeError: only integer scalar arrays can be converted to a scalar index

In [379]:
## Extracting MFCC < Mel frequency celstral coeffecients

In [380]:
import librosa.display
import IPython.display as ipd
import librosa
import os

In [77]:
!python test.py --file "recorded_audio_female6.wav"

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                         

In [24]:
import IPython.display as ipd

In [39]:
ipd.Audio("recorded_audio_female6.wav")

# CNN Network?

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
input_shape =128
if tf.test.is_gpu_available():
    tf.config.set_soft_device_placement(True)
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(physical_devices), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

def create_cnn_model(input_shape):
    model = Sequential()

    # Convolutional layers
    model.add(Conv1D(32, 3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(64, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(128, 3, activation='relu'))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(256, 3, activation='relu'))
    model.add(MaxPooling1D(2))

    # Flatten layer
    model.add(Flatten())

    # Dense layers
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Print summary of the model
    model.summary()

    return model


1 Physical GPUs, 1 Logical GPUs


In [49]:
# Assuming your input data is stored in a variable called 'X'
# Reshape the input data to match the expected shape
X = np.reshape(X, (X.shape[0], 128, 1))

# Split the data into training, validation, and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[0], 'GPU')

# Construct the model
model = create_cnn_model((128, 1))




# Use TensorBoard to view metrics
tensorboard = TensorBoard(log_dir="logs")

# Define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100

# Train the model using the training set and validate using the validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size,
          validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# Save the model to a file
model.save("results_cnn1/model.h5")

# Evaluate the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_13 (Conv1D)          (None, 126, 32)           128       
                                                                 
 max_pooling1d_12 (MaxPooli  (None, 63, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_14 (Conv1D)          (None, 61, 64)            6208      
                                                                 
 max_pooling1d_13 (MaxPooli  (None, 30, 64)            0         
 ng1D)                                                           
                                                                 
 conv1d_15 (Conv1D)          (None, 28, 128)           24704     
                                                                 
 max_pooling1d_14 (MaxPooli  (None, 14, 128)          

848/848 [==============================] - 12s 14ms/step - loss: 0.4008 - accuracy: 0.8176 - val_loss: 0.2667 - val_accuracy: 0.9002
Epoch 2/100
848/848 [==============================] - 12s 14ms/step - loss: 0.2540 - accuracy: 0.9002 - val_loss: 0.2190 - val_accuracy: 0.9177
Epoch 3/100
848/848 [==============================] - 12s 14ms/step - loss: 0.2209 - accuracy: 0.9161 - val_loss: 0.2051 - val_accuracy: 0.9256
Epoch 4/100
848/848 [==============================] - 12s 14ms/step - loss: 0.2020 - accuracy: 0.9235 - val_loss: 0.1928 - val_accuracy: 0.9280
Epoch 5/100
848/848 [==============================] - 11s 13ms/step - loss: 0.1860 - accuracy: 0.9302 - val_loss: 0.1857 - val_accuracy: 0.9310
Epoch 6/100
848/848 [==============================] - 12s 14ms/step - loss: 0.1761 - accuracy: 0.9344 - val_loss: 0.1871 - val_accuracy: 0.9315
Epoch 7/100
848/848 [==============================] - 12s 14ms/step - loss: 0.1649 - accuracy: 0.9383 - val_loss: 0.1824 - val_accuracy: 0.93

/Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Loss: 0.1610
Accuracy: 94.26%


In [59]:
# Reshape the input data to match the expected shape
X_train = data["X_train"].reshape(data["X_train"].shape[0], data["X_train"].shape[1], 1)
X_valid = data["X_valid"].reshape(data["X_valid"].shape[0], data["X_valid"].shape[1], 1)
X_test = data["X_test"].reshape(data["X_test"].shape[0], data["X_test"].shape[1], 1)

# Train the model using the reshaped data
model.fit(X_train, data["y_train"], epochs=epochs, batch_size=batch_size, validation_data=(X_valid, data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# Evaluate the model using the reshaped testing data
loss, accuracy = model.evaluate(X_test, data["y_test"], verbose=0)
model.save("results_cnn2/model.h5")



Epoch 1/100
121/848 [===>..........................] - ETA: 7s - loss: 0.8135 - accuracy: 0.6628

848/848 [==============================] - 10s 11ms/step - loss: 0.5481 - accuracy: 0.7822 - val_loss: 0.3663 - val_accuracy: 0.8563
Epoch 2/100
848/848 [==============================] - 9s 10ms/step - loss: 0.3940 - accuracy: 0.8419 - val_loss: 0.3485 - val_accuracy: 0.8624
Epoch 3/100
848/848 [==============================] - 9s 10ms/step - loss: 0.3633 - accuracy: 0.8593 - val_loss: 0.3133 - val_accuracy: 0.8818
Epoch 4/100
848/848 [==============================] - 9s 11ms/step - loss: 0.3452 - accuracy: 0.8659 - val_loss: 0.2977 - val_accuracy: 0.8802
Epoch 5/100
848/848 [==============================] - 9s 10ms/step - loss: 0.3279 - accuracy: 0.8710 - val_loss: 0.2879 - val_accuracy: 0.8803
Epoch 6/100
848/848 [==============================] - 9s 11ms/step - loss: 0.3191 - accuracy: 0.8762 - val_loss: 0.2885 - val_accuracy: 0.8883
Epoch 7/100
848/848 [==============================] - 9s 11ms/step - loss: 0.3112 - accuracy: 0.8800 - val_loss: 0.2752 - val_accuracy: 0.8946
Epo

In [60]:
model.save("results_cnn2/model.h5")

/Users/nanda/miniforge3/envs/env_tensor/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [61]:
import numpy as np
import pyaudio
import wave
import librosa

def record_to_file(file_name, duration=5, sample_rate=44100):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=CHUNK)

    frames = []
    for i in range(0, int(sample_rate / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    print('#######')
    wf.close()

def extract_feature(file_name, **kwargs):
    X, sample_rate = librosa.load(file_name)

    features = []
    if kwargs.get("mel"):
        mel = librosa.feature.melspectrogram(y=X, sr=sample_rate)
        print(len(mel))
        features.append(mel)
    if kwargs.get("chroma"):
        chroma = librosa.feature.chroma_stft(y=X, sr=sample_rate)
        features.append(chroma)
    if kwargs.get("contrast"):
        contrast = librosa.feature.spectral_contrast(y=X, sr=sample_rate)
        features.append(contrast)
    if kwargs.get("tonnetz"):
        tonnetz = librosa.feature.tonnetz(y=X, sr=sample_rate)
        features.append(tonnetz)

    if len(features) > 0:
        features = np.concatenate(features, axis=0)

    return features

# Example usage

duration = 8

# Record audio
#record_to_file(file, duration=duration)

# Extract features and reshape
#features = extract_feature(file, mel=True).reshape(-1, 1)

# Perform further processing on the extracted features
# ...



# Rest of the code remains the same...


if __name__ == "__main__":
    # load the saved model (after training)
    # model = pickle.load(open("result/mlp_classifier.model", "rb"))
    from my_utils import load_data, split_data, create_model
    import argparse
    parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
    parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
    args = parser.parse_args()
    #file = "New Recording 17.wav"
    #file = args.file
    # construct the model
    model = create_cnn_model((128,))
    # load the saved/trained weights
    #file = 'recorded_audio_female6.wav"
    model.load_weights("results_cnn2/model.h5")
   # if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please start speaking...")
        # wait for user to press enter to start recording
        #input("Press Enter to start recording...")
        # specify the duration of the recording (in seconds)
        #duration = 5
        # record the file
        #record_to_file(file, duration=duration)
    # extract features and reshape it
    #if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please talk")
        # put the file name here
        #file = "test.wav"
        # record the file (start talking)
        #record_to_file("/Users/nanda/Documents/mini_project")
    
    #.reshape(1, -1)

# Assuming you have extracted the features into the 'features' variable with shape (None, 27648)

# Reshape the features to match the expected input shape
   # reshaped_features = np.reshape(features, (features.shape[0], 128))

# Now 'reshaped_features' has the shape (None, 128) which matches the model's expected input shape
    file =("/Users/nanda/Documents/mini_project/recordedmale15.wav")
    features = extract_feature(file, mel=True)
    print((features.shape), '******')
    features = features.reshape(-1, 128)
    print(len(features.shape), '******')
    print(type(features))
    # predict the gender!
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    print(f"{file}")
    print("Result:", gender)
    print(f"Probabilities:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")
    print(file)

ValueError: Input 0 of layer "conv1d_22" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 128)

In [50]:
record_to_file("/Users/nanda/Documents/mini_project/recordedmale15.wav")

#######


In [66]:
X.shape

(66938, 128, 1)

In [62]:
import numpy as np
from my_utils import load_data, split_data

# Assuming your input data is stored in a variable called 'X'
# Reshape the input data to match the expected shape
X = np.reshape(X, (X.shape[0], 128, 1))

# Split the data into training, validation, and testing sets
data = split_data(X, y, test_size=0.1, valid_size=0.1)

# Construct the model
model = create_cnn_model((128, 1))

# Use TensorBoard to view metrics
tensorboard = TensorBoard(log_dir="new_logs")

# Define early stopping to stop training after 5 epochs of not improving
early_stopping = EarlyStopping(mode="min", patience=5, restore_best_weights=True)

batch_size = 64
epochs = 100

# Train the model using the training set and validate using the validation set
model.fit(data["X_train"], data["y_train"], epochs=epochs, batch_size=batch_size,
          validation_data=(data["X_valid"], data["y_valid"]),
          callbacks=[tensorboard, early_stopping])

# Save the model to a file
model.save("results_cnn1/model.h5")

# Evaluate the model using the testing set
print(f"Evaluating the model using {len(data['X_test'])} samples...")
loss, accuracy = model.evaluate(data["X_test"], data["y_test"], verbose=0)
print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy * 100:.2f}%")


Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_23 (Conv1D)          (None, 126, 32)           128       
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 63, 32)            0         
 ng1D)                                                           
                                                                 
 conv1d_24 (Conv1D)          (None, 61, 64)            6208      
                                                                 
 max_pooling1d_17 (MaxPooli  (None, 30, 64)            0         
 ng1D)                                                           
                                                                 
 conv1d_25 (Conv1D)          (None, 28, 128)           24704     
                                                                 
 max_pooling1d_18 (MaxPooli  (None, 14, 128)         

In [63]:
import numpy as np
import pyaudio
import wave
import librosa

def record_to_file(file_name, duration=5, sample_rate=44100):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=CHUNK)

    frames = []
    for i in range(0, int(sample_rate / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    print('#######')
    wf.close()

def extract_feature(file_name, **kwargs):
    X, sample_rate = librosa.load(file_name)

    features = []
    if kwargs.get("mel"):
        mel = librosa.feature.melspectrogram(y=X, sr=sample_rate)
        print(len(mel))
        features.append(mel)
    if kwargs.get("chroma"):
        chroma = librosa.feature.chroma_stft(y=X, sr=sample_rate)
        features.append(chroma)
    if kwargs.get("contrast"):
        contrast = librosa.feature.spectral_contrast(y=X, sr=sample_rate)
        features.append(contrast)
    if kwargs.get("tonnetz"):
        tonnetz = librosa.feature.tonnetz(y=X, sr=sample_rate)
        features.append(tonnetz)

    if len(features) > 0:
        features = np.concatenate(features, axis=0)

    return features

# Example usage

duration = 8

# Record audio
#record_to_file(file, duration=duration)

# Extract features and reshape
#features = extract_feature(file, mel=True).reshape(-1, 1)

# Perform further processing on the extracted features
# ...



# Rest of the code remains the same...


if __name__ == "__main__":
    # load the saved model (after training)
    # model = pickle.load(open("result/mlp_classifier.model", "rb"))
    from my_utils import load_data, split_data, create_model
    import argparse
    parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
    parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
    args = parser.parse_args()
    #file = "New Recording 17.wav"
    #file = args.file
    # construct the model
    model = create_cnn_model((128,))
    # load the saved/trained weights
    #file = 'recorded_audio_female6.wav"
    model.load_weights("results_cnn2/model.h5")
   # if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please start speaking...")
        # wait for user to press enter to start recording
        #input("Press Enter to start recording...")
        # specify the duration of the recording (in seconds)
        #duration = 5
        # record the file
        #record_to_file(file, duration=duration)
    # extract features and reshape it
    #if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please talk")
        # put the file name here
        #file = "test.wav"
        # record the file (start talking)
        #record_to_file("/Users/nanda/Documents/mini_project")
    
    #.reshape(1, -1)

# Assuming you have extracted the features into the 'features' variable with shape (None, 27648)

# Reshape the features to match the expected input shape
   # reshaped_features = np.reshape(features, (features.shape[0], 128))

# Now 'reshaped_features' has the shape (None, 128) which matches the model's expected input shape
    file =("/Users/nanda/Documents/mini_project/recordedmale15.wav")
    features = extract_feature(file, mel=True)
    print((features.shape), '******')
    features = features.reshape(-1, 128)
    print(len(features.shape), '******')
    print(type(features))
    # predict the gender!
    male_prob = model.predict(features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    print(f"{file}")
    print("Result:", gender)
    print(f"Probabilities:     Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")
    print(file)

ValueError: Input 0 of layer "conv1d_27" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 128)

In [74]:
import numpy as np
import pyaudio
import wave
import librosa

def record_to_file(file_name, duration=5, sample_rate=44100):
    CHUNK = 1024
    FORMAT = pyaudio.paInt16
    CHANNELS = 1

    audio = pyaudio.PyAudio()

    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=CHUNK)

    frames = []
    for i in range(0, int(sample_rate / CHUNK * duration)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    audio.terminate()

    wf = wave.open(file_name, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(sample_rate)
    wf.writeframes(b''.join(frames))
    print('#######')
    wf.close()

def extract_feature(file_name, **kwargs):
    X, sample_rate = librosa.load(file_name)

    features = []
    if kwargs.get("mel"):
        mel = librosa.feature.melspectrogram(y=X, sr=sample_rate)
        print(len(mel))
        features.append(mel)
    if kwargs.get("chroma"):
        chroma = librosa.feature.chroma_stft(y=X, sr=sample_rate)
        features.append(chroma)
    if kwargs.get("contrast"):
        contrast = librosa.feature.spectral_contrast(y=X, sr=sample_rate)
        features.append(contrast)
    if kwargs.get("tonnetz"):
        tonnetz = librosa.feature.tonnetz(y=X, sr=sample_rate)
        features.append(tonnetz)

    if len(features) > 0:
        features = np.concatenate(features, axis=0)

    return features

# Example usage

duration = 8

# Record audio
#record_to_file(file, duration=duration)

# Extract features and reshape
#features = extract_feature(file, mel=True).reshape(-1, 1)

# Perform further processing on the extracted features
# ...



# Rest of the code remains the same...


if __name__ == "__main__":
    # load the saved model (after training)
    # model = pickle.load(open("result/mlp_classifier.model", "rb"))
    from my_utils import load_data, split_data, create_model
    import argparse
    parser = argparse.ArgumentParser(description="""Gender recognition script, this will load the model you trained, 
                                    and perform inference on a sample you provide (either using your voice or a file)""")
    parser.add_argument("-f", "--file", help="The path to the file, preferred to be in WAV format")
    args = parser.parse_args()
    #file = "New Recording 17.wav"
    #file = args.file
    # construct the model
    model = create_model()
    # load the saved/trained weights
    file = "New Recording 17.wav"
    model.load_weights("results_try2/model.h5")
   # if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please start speaking...")
        # wait for user to press enter to start recording
        #input("Press Enter to start recording...")
        # specify the duration of the recording (in seconds)
        #duration = 5
        # record the file
        #record_to_file(file, duration=duration)
    # extract features and reshape it
    #if not file or not os.path.isfile(file):
        # if file not provided, or it doesn't exist, use your voice
        #print("Please talk")
        # put the file name here
        #file = "test.wav"
        # record the file (start talking)
        #record_to_file("/Users/nanda/Documents/mini_project")
    
    #.reshape(1, -1)

# Assuming you have extracted the features into the 'features' variable with shape (None, 27648)

# Reshape the features to match the expected input shape
   # reshaped_features = np.reshape(features, (features.shape[0], 128))

# Now 'reshaped_features' has the shape (None, 128) which matches the model's expected input shape
    file =("recorded_audio_male2.wav")
    features = extract_feature(file,mel=True,mfcc=True,chroma=True,contrast=True,tonnetz=True)
    print((features.shape), '#####')
    import numpy as np

# Assuming you have an array of any length
      # Example array, you can replace it with your own data

  

# Assuming you have an array of any length

    desired_length = 128

#reshaped_features = np.random.rand(32, 216)  # Example reshaped features, replace it with your own data

    desired_length = 128

# Truncate or pad the reshaped features to the desired length
    reshaped_features = reshaped_features[:, :desired_length]

# Add an extra dimension to match the model's expected input shape
    reshaped_features = np.expand_dims(reshaped_features, axis=0)

    print(reshaped_features.shape)  # Output: (1, 32, 128)ngth]

    print(reshaped_features.shape)  # Output: (128)
    reshaped_features = np.random.rand(32, 1, 128, 128)  # Example reshaped features, replace it with your own data

    desired_length = 128

# Reshape the features to (None, 128)
    reshaped_features = np.reshape(reshaped_features, (reshaped_features.shape[0], desired_length))
    padded_features = np.pad(array, (0, max(0, desired_length - len(reshaped_features))), mode='constant')

# Truncate or pad the array to the desired length
    reshaped_features= padded_features[:desired_length]

# Reshape the array to have shape (None, 128)
    reshaped_features = np.expand_dims(reshaped_features, axis=0)

    print(reshaped_features.shape) 

    #features = features.reshape(-1, 128)
    print(len(reshaped_features.shape), '******')
    print(type(reshaped_features))
    male_prob = model.predict(reshaped_features)[0][0]
    female_prob = 1 - male_prob
    gender = "male" if male_prob > female_prob else "female"
    # show the result!
    print(f"{file}")
    print("Result:", gender)
    print(f"Probabilities Male: {male_prob*100:.2f}%    Female: {female_prob*100:.2f}%")
    print(file)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_74 (Dense)            (None, 256)               33024     
                                                                 
 dropout_60 (Dropout)        (None, 256)               0         
                                                                 
 dense_75 (Dense)            (None, 256)               65792     
                                                                 
 dropout_61 (Dropout)        (None, 256)               0         
                                                                 
 dense_76 (Dense)            (None, 128)               32896     
                                                                 
 dropout_62 (Dropout)        (None, 128)               0         
                                                                 
 dense_77 (Dense)            (None, 128)             

ValueError: cannot reshape array of size 524288 into shape (32,128)